In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer



In [2]:
data=pd.read_csv("C:/Users/JOSHWIN ISAC/OneDrive/Documents/GitHub/Quantum-Computing-project/Quantum_NLP/IMDB_dataset.csv")

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
label_encoder=LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string

[nltk_data] Downloading package stopwords to C:\Users\JOSHWIN
[nltk_data]     ISAC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\JOSHWIN
[nltk_data]     ISAC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
#remove url
def remove_url(text):
  url_pattern=re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'',text)

#Text to lower
def to_lower(text):
  return text.lower()

#remove punctuation
def remove_punc(text):
  translator=str.maketrans('','',string.punctuation)
  return text.translate(translator)

#remove stopwords
def remove_stopwords(text):
  stop_words=set(stopwords.words('english'))
  words=text.split()
  filtered_words=[word for word in words if word not in stop_words]
  return ' '.join(filtered_words)

#remove numbers
def remove_numbers(text):
  return re.sub(r'\d+', '', text)

# Lemmatize text
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

#remove symbols
def remove_symbols(text):
  return re.sub(r'[^\w\s]', '', text)

In [8]:
def preprocess_text(text):
  text=remove_url(text)
  text=to_lower(text)
  text=remove_punc(text)
  text=remove_stopwords(text)
  text=remove_numbers(text)
  text=lemmatize_text(text)
  text=remove_symbols(text)
  return text

#calling the funtion preprocess
data['review']=data['review'].apply(preprocess_text)
data.head(5)

,review,sentiment
0,one reviewer mentioned watching oz episode you...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there family little boy jake think t...,0
4,petter matteis love time money visually stunni...,1


In [9]:
#show the comments after preprocessing in df
data['Cleaned_reviews']=data['review'].apply(remove_url).apply(to_lower).apply(remove_punc).apply(remove_stopwords).apply(remove_numbers).apply(lemmatize_text)
data.head(5)

,review,sentiment,Cleaned_reviews
0,one reviewer mentioned watching oz episode you...,1,one reviewer mentioned watching oz episode you...
1,wonderful little production br br filming tech...,1,wonderful little production br br filming tech...
2,thought wonderful way spend time hot summer we...,1,thought wonderful way spend time hot summer we...
3,basically there family little boy jake think t...,0,basically family little boy jake think zombie ...
4,petter matteis love time money visually stunni...,1,petter matteis love time money visually stunni...


In [10]:
vectorizer=TfidfVectorizer(max_features=100)
features = vectorizer.fit_transform(data['Cleaned_reviews']).toarray()
y=data['sentiment']
print(features)

[[0.         0.         0.         ... 0.1144768  0.         0.        ]
 [0.         0.16063599 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.16557483 0.20089626]
 ...
 [0.1572673  0.         0.         ... 0.13161938 0.         0.        ]
 [0.         0.         0.         ... 0.16156238 0.         0.        ]
 [0.         0.         0.         ... 0.11756074 0.         0.        ]]


In [11]:
X_train,X_test,y_train,y_test=train_test_split(features,y,test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classifier =LogisticRegression()
classifier.fit(X_train,y_train)

# Evaluate the model
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy (Classical Model): {accuracy}")

Accuracy (Classical Model): 0.7384


In [ ]:
from sklearn.svm import SVC
SVM_classifier=SVC()
SVM_classifier.fit(X_train,y_train)


prediction_SVM=SVM_classifier.predict(X_test)
accuracy_SVM=accuracy_score(y_test,prediction_SVM)

print(f"Accuracy (Classical Model): {accuracy_SVM}")

In [ ]:
from qiskit_machine_learning.algorithms import VQC
from qiskit.circuit.library import TwoLocal


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Normalize features to [0, 1] for quantum encoding
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Select a subset of features for quantum encoding (limited to the number of qubits)
n_qubits = 4
quantum_features = scaled_features[:, :2**n_qubits]


In [1]:
import qiskit